In [26]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [114]:
dir = os.getcwd()
train = pd.read_csv(dir + "/data/train_data.csv")
test = pd.read_csv(dir + "/data/test_data.csv")
valid = pd.read_csv(dir + "/data/valid_data.csv")

train_fil = train.fillna(0.5)
test_fil = test.fillna(0.5)
valid_fil = valid.fillna(0.5)

In [73]:
def Agediv(age):
    if age<1:
        return 1
    elif age<5:
        return 2
    elif age<10:
        return 3
    elif age<15:
        return 4
    elif age<20:
        return 5
    elif age<25:
        return 6
    elif age<30:
        return 7
    elif age<35:
        return 8
    elif age<40:
        return 9
    elif age<45:
        return 10
    elif age<50:
        return 11
    elif age<55:
        return 12
    elif age<60:
        return 13
    elif age<65:
        return 14
    elif age<70:
        return 15
    elif age<75:
        return 16
    elif age<80:
        return 17
    elif age<85:
        return 18
    else:
        return 19



In [28]:

lifetable = pd.read_csv("/Users/ichang-ug/Documents/GitHub/bitamin_ChurnProject/changwook/data/lifetable.csv", header=1)
lifetable = lifetable.iloc[:,:5]
lifetable

,Indicator,Age Group,Both sexes,Male,Female
0,nMx - age-specific death rate between ages x a...,<1 year,0.006805,0.007220,0.006339
1,nMx - age-specific death rate between ages x a...,1-4 years,0.000286,0.000302,0.000268
2,nMx - age-specific death rate between ages x a...,5-9 years,0.000185,0.000195,0.000173
3,nMx - age-specific death rate between ages x a...,10-14 years,0.000194,0.000242,0.000138
4,nMx - age-specific death rate between ages x a...,15-19 years,0.000360,0.000482,0.000219
...,...,...,...,...,...
128,ex - expectation of life at age x,65-69 years,17.007469,15.318173,18.801657
129,ex - expectation of life at age x,70-74 years,13.237153,11.730069,14.791321
130,ex - expectation of life at age x,75-79 years,9.945075,8.646179,11.224150
131,ex - expectation of life at age x,80-84 years,7.062694,5.943793,8.079565


In [29]:
deathproba = lifetable[lifetable["Indicator"]=="nqx - probability of dying between ages x and x+n"].copy()
deathproba

,Indicator,Age Group,Both sexes,Male,Female
19,nqx - probability of dying between ages x and x+n,<1 year,0.006764,0.007174,0.006303
20,nqx - probability of dying between ages x and x+n,1-4 years,0.001144,0.001209,0.001072
21,nqx - probability of dying between ages x and x+n,5-9 years,0.000925,0.000976,0.000865
22,nqx - probability of dying between ages x and x+n,10-14 years,0.000970,0.001209,0.000691
23,nqx - probability of dying between ages x and x+n,15-19 years,0.001797,0.002407,0.001096
24,nqx - probability of dying between ages x and x+n,20-24 years,0.002811,0.003951,0.001523
25,nqx - probability of dying between ages x and x+n,25-29 years,0.003053,0.004367,0.001608
26,nqx - probability of dying between ages x and x+n,30-34 years,0.004240,0.006103,0.002259
27,nqx - probability of dying between ages x and x+n,35-39 years,0.006188,0.008829,0.003410
28,nqx - probability of dying between ages x and x+n,40-44 years,0.009315,0.013058,0.005384


In [163]:
def interest(r, num):
    """
    Args:
        r (float): interest rate
        num (int): how many times paid
    """
    sum = 1
    for n in range(num):
        sum = (1+r)*sum 
    return sum

In [161]:
def findAx(data, deathproba, i, ini):
    """
    this is a function to find Ax of every individuals when 5qx data is given, and make it into a pandas series.
        Args:
            data (dataframe): requires age, gender of each indiviuals
            deathproba (dataframe): 5qx data in 5yrs period for male, female and both sexes.
            i (float): geometric series of avg interest rate to 5yrs (because our data is in 5yrs period)
            ini (float): geometric series of average interest rate to 2.5yrs (initial Ax)

    """
    Ax_list = []
    for ind, age in enumerate(data["I2"]):
        ran = Agediv(age)-1
        gen = data["I1"].iloc[ind]

        if (gen==0.0):
            prob_copy = deathproba.loc[:,"Female"].iloc[ran:].copy()
        elif(gen ==1.0):
            prob_copy = deathproba.loc[:,"Male"].iloc[ran:].copy()
        elif(gen == 0.5):
            prob_copy = deathproba.loc[:,"Both sexes"].iloc[ran:].copy()
            
        aqx_list = []
        Ax = 0
        for a in range(len(prob_copy)):
            if(a==0):
                aqx = prob_copy.iloc[a]
            else:
                aqx = 1
                for b in range(a):
                    aqx *= (1-prob_copy.iloc[b])
                aqx *= prob_copy.iloc[a]
            aqx_list.append(aqx)
        for num, aqx in enumerate(aqx_list):
            Ax += ini*interest(i,num+1)*aqx

        Ax_list.append(Ax)
    
    return Ax_list
        
    

In [159]:
# test purpose
# a = [0.007506399, 0.012568696794625356, 0.019938001886493584, 0.03252870019129668, 0.05308280494572649, 0.09128655770408828, 0.1373359616916808, 0.20105508100835345, 0.4446977967777354]
# sum=0
# for num in a:
#     sum+=num
# print(sum)

In [164]:
findAx(train_fil, deathproba, 0.342, 4.142964)

[24.38612078990632,
 43.172370725138194,
 63.77403135728726,
 103.2852893408546,
 47.908020407230026,
 77.12856953701608,
 138.39494610372026,
 7.974554736267805,
 10.889028742533815,
 103.2852893408546,
 57.655324600058606,
 20.784062103596618,
 185.45160100777127,
 77.12856953701608,
 103.2852893408546,
 138.39494610372026,
 47.908020407230026,
 10.889028742533815,
 24.38612078990632,
 9.8179059366062,
 138.39494610372026,
 85.0963559225051,
 14.094918178847745,
 12.419998923409107,
 10.889028742533815,
 32.382146497593226,
 18.46283465970283,
 77.12856953701608,
 14.094918178847745,
 7.974554736267805,
 103.2852893408546,
 32.382146497593226,
 103.2852893408546,
 9.8179059366062,
 113.724923096121,
 113.724923096121,
 32.382146497593226,
 15.963851436683242,
 18.46283465970283,
 12.419998923409107,
 20.784062103596618,
 6.614929930989051,
 43.172370725138194,
 32.382146497593226,
 77.12856953701608,
 12.419998923409107,
 20.784062103596618,
 24.38612078990632,
 9.8179059366062,
 9.8

In [156]:
a = [0.007506399, 0.012568696794625356, 0.019938001886493584, 0.03252870019129668, 0.05308280494572649, 0.09128655770408828, 0.1373359616916808, 0.20105508100835345, 0.4446977967777354]
sum=0
for num in a:
    sum+=num
print(sum)

1.0


In [ ]:
deathproba.loc[:,"Female"].iloc[1:]

20    0.001072
21    0.000865
22    0.000691
23    0.001096
24    0.001523
25    0.001608
26    0.002259
27    0.003410
28    0.005384
29    0.007506
30    0.012664
31    0.020346
32    0.033885
33    0.057235
34    0.104402
35    0.175377
36    0.311350
37    1.000000
Name: Female, dtype: float64

In [70]:
i = 0.342
sum = 0
sum = (1+i)*sum+1
sum = (1+i)*sum +1
sum = (1+i)*sum +1
sum


4.142964